In [28]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from lime import lime_tabular

In [39]:
train = pd.read_csv ('https://raw.githubusercontent.com/claradelatorre/Test2ML-Classifier/main/train.csv')
test = pd.read_csv ('https://raw.githubusercontent.com/claradelatorre/Test2ML-Classifier/main/test.csv')